<a href="https://colab.research.google.com/github/Chenyuju/chen/blob/master/Regression(Abalone_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [76]:
!apt-get install openjdk-11-jdk-headless > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer, VectorIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
import findspark
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [80]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

abalone_data = spark.read.format('com.databricks.spark.csv').\
                              options(header='true', \
                              inferschema='true') \
                .load("/content/sample_data/abalone.csv",header=True);

abalone_data.show(10)

+---+------+--------+------+------------+--------------+--------------+------------+-----+
|Sex|Length|Diameter|Height|Whole weight|Shucked weight|Viscera weight|Shell weight|Rings|
+---+------+--------+------+------------+--------------+--------------+------------+-----+
|  1| 0.455|   0.365| 0.095|       0.514|        0.2245|         0.101|        0.15|   15|
|  1|  0.35|   0.265|  0.09|      0.2255|        0.0995|        0.0485|        0.07|    7|
|  2|  0.53|    0.42| 0.135|       0.677|        0.2565|        0.1415|        0.21|    9|
|  1|  0.44|   0.365| 0.125|       0.516|        0.2155|         0.114|       0.155|   10|
|  0|  0.33|   0.255|  0.08|       0.205|        0.0895|        0.0395|       0.055|    7|
|  0| 0.425|     0.3| 0.095|      0.3515|         0.141|        0.0775|        0.12|    8|
|  2|  0.53|   0.415|  0.15|      0.7775|         0.237|        0.1415|        0.33|   20|
|  2| 0.545|   0.425| 0.125|       0.768|         0.294|        0.1495|        0.26|   16|

In [81]:
def transData(data):
  return data.rdd.map(lambda r: [Vectors.dense(r[1:]),r[0]]).toDF(['features','label'])

transformed = transData(abalone_data)
trainingData, testData = transformed.randomSplit([0.7, 0.3], seed = 100)
transformed.sample(False, 0.1, 123).show(10)

#abalone_data.count(),trainingData.count(),testData.count()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.535,0.405,0.14...|    2|
|[0.705,0.55,0.2,1...|    2|
|[0.47,0.37,0.12,0...|    1|
|[0.45,0.345,0.105...|    1|
|[0.475,0.39,0.12,...|    2|
|[0.625,0.465,0.14...|    1|
|[0.46,0.355,0.13,...|    2|
|[0.36,0.265,0.095...|    0|
|[0.5,0.4,0.125,0....|    2|
|[0.345,0.255,0.09...|    1|
+--------------------+-----+
only showing top 10 rows



In [0]:
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(transformed)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(transformed)
#labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")

In [83]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

pipeline_dt = Pipeline(stages=[featureIndexer, dt])
model_dt = pipeline_dt.fit(trainingData)
#predictions_dt = model_dt.transform(testData)

#accuracy_dt = evaluator.evaluate(predictions_dt)

IllegalArgumentException: ignored